In [266]:
# PT -> Projeto de otimização com a biblioteca Pulp do Python
# EN -> Project of optimization using the lib Pulp from Python


#Inspired by the conference of Anna Nicanorova at PyData NYC 2015
#(https://www.youtube.com/watch?v=7yZ5xxdkTb8)

In [305]:
# PT -> Baixar todas as bibliotecas que serao utilizadas no projeto
# EN -> Download all the libs for the project 

from pulp import *
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import matplotlib.pyplot as plt

In [268]:
# PT -> Estabelecer conexao pagina web (site goodreads) e criar objeto beautifulsoup para cada genero de livro escolhido
# EN -> Establish connection with website page (goodreads) and create beatifulsoup object for each book genre 


#Durante todo o projeto as seguintes letras serão utilizadas para se referir a cada um dos gêneros de livros escolhido
#During all the project the following letters are going to be used to refer to each chosen genre
#b - business     c - Best Crime & Mystery Books         g-german

#business
url_b = 'https://www.goodreads.com/shelf/show/business'
response_b = urlopen(url_b)
html_b = response_b.read()
soup_b = BeautifulSoup(html_b)


#crime
url_c = 'https://www.goodreads.com/list/show/11.Best_Crime_Mystery_Books'
response_c = urlopen(url_c)
html_c = response_c.read()
soup_c = BeautifulSoup(html_c)



#german
url_g = 'https://www.goodreads.com/list/show/41421.Books_we_read_in_a_German_Gymnasium_'
response_g = urlopen(url_g)
html_g = response_g.read()
soup_g = BeautifulSoup(html_g)

In [269]:
# PT -> Realizar um filtro com base em uma tag html específica, 
# EN -> Read the html and filter an specific tag 


#business
livros_b = soup_b.findAll("div", {"class" : "elementList" })
livros_b



#crime
livros_c = soup_c.findAll("td", {"width" : "100%" })
livros_c


#crime
livros_g = soup_g.findAll("td", {"width" : "100%" })
livros_g

[<td valign="top" width="100%">
 <a class="bookTitle" href="/book/show/1095059.Faust" itemprop="url">
 <span aria-level="4" itemprop="name" role="heading">Faust: Der Tragödie Erster Teil</span>
 </a> <br/>
 <span class="by">by</span>
 <span itemprop="author" itemscope="" itemtype="http://schema.org/Person">
 <div class="authorName__container">
 <a class="authorName" href="https://www.goodreads.com/author/show/285217.Johann_Wolfgang_von_Goethe" itemprop="url"><span itemprop="name">Johann Wolfgang von Goethe</span></a>
 </div>
 </span>
 <br/>
 <div>
 <span class="greyText smallText uitext">
 <span class="minirating"><span class="stars staticStars notranslate"><span class="staticStar p10" size="12x12"></span><span class="staticStar p10" size="12x12"></span><span class="staticStar p10" size="12x12"></span><span class="staticStar p6" size="12x12"></span><span class="staticStar p0" size="12x12"></span></span> 3.93 avg rating — 60,447 ratings</span>
 </span>
 </div>
 <div style="margin-top: 5

In [270]:
# PT -> Criar lista de dicionários com todas as informações desejadas  dos livros de business (titulo, autor, nota, total de avaliacoes, url, numero paginas)
# EN -> Create a list of dicts with all the desired information about the business books (title, author, rating, total ratings, url, number of pages)



itens_b = []

for livro in livros_b[0:50]:
    dicionario_livros_b = {}

    #Titulo
    dicionario_livros_b['titulo'] = livro.find("a", {"class" : "leftAlignedImage" }).get('title')
    

    #Autor
    dicionario_livros_b['autor'] = livro.find("a", {"class" : "authorName"}).get_text()
    
    #Nota media das avaliacoes 
    dicionario_livros_b['nota'] = float(livro.find("span", {"class" : "greyText smallText"}).get_text().split('—')[0].split()[2])
    
    #Total de avaliacoes 
    dicionario_livros_b['total_avaliacoes'] = int(livro.find("span", {"class" : "greyText smallText"}).get_text().split('—')[1].split()[0].replace(',',""))
    
    #Url
    dicionario_livros_b['url'] = 'https://www.goodreads.com'+livro.find('a', {'class':"bookTitle"}).get('href')
    
    
    #Paginas business
    url_b_2 = dicionario_livros_b['url']
    response_b_2 = urlopen(url_b_2)
    html_b_2 = response_b_2.read()
    soup_b_2 = BeautifulSoup(html_b_2)
    if soup_b_2.find("span", {"itemprop" : "numberOfPages" }):
        dicionario_livros_b['numero_paginas'] = int(soup_b_2.find("span", {"itemprop" : "numberOfPages" }).get_text().split()[0])
    
    
    itens_b.append(dicionario_livros_b)

In [271]:
# PT -> Criar lista de dicionários com todas as informações desejadas  dos livros de crimes (titulo, autor, nota, total de avaliacoes, url, numero paginas)
# EN -> Create a list of dicts with all the desired information about the crime books (title, author, rating, total ratings, url, number of pages)


itens_c = []

for livro in livros_c:
    dicionario_livros_c = {}

    #Titulo - Title
    dicionario_livros_c['titulo'] = livro.find("span", {"itemprop" : "name" }).get_text()

    #Autor - Author
    dicionario_livros_c['autor'] = livro.find("a", {"class" : "authorName"}).get_text()
    
    #Nota media das avaliacoes  - rating grade 
    dicionario_livros_c['nota'] = float(livro.find("span", {"class" : "minirating"}).get_text().split('-')[0].split()[0])
    
    #Total de avaliacoes - total ratings
    dicionario_livros_c['total_avaliacoes'] = int((livro.find("a", {"href" : "#"}).get_text().split()[1]).replace(',',''))
    
    #Url
    dicionario_livros_c['url'] = 'https://www.goodreads.com'+livro.find('a', {'class':"bookTitle"}).get('href')

   
    
    #Paginas crimes - crime pages
    url_c_2 = dicionario_livros_c['url']
    response_c_2 = urlopen(url_c_2)
    html_c_2 = response_c_2.read()
    soup_c_2 = BeautifulSoup(html_c_2)
    if soup_c_2.find("span", {"itemprop" : "numberOfPages" }):
        dicionario_livros_c['numero_paginas'] = int(soup_c_2.find("span", {"itemprop" : "numberOfPages" }).get_text().split()[0])
    
    
    itens_c.append(dicionario_livros_c)


In [272]:
# PT -> Criar lista de dicionários com todas as informações desejadas  dos livros de alemao (titulo, autor, nota, total de avaliacoes, url, numero paginas)
# EN -> Create a list of dicts with all the desired information about the german languague books (title, author, rating grade, total ratings, url, number of pages)


itens_g = []

for livro in livros_g:
    dicionario_livros_g = {}

    

    #Titulo - Title
    dicionario_livros_g['titulo'] = livro.find("span", {"itemprop" : "name" }).get_text()

    #Autor - Author
    dicionario_livros_g['autor'] = livro.find("a", {"class" : "authorName"}).get_text()

    #Nota media das avaliacoes - rating grade 
    dicionario_livros_g['nota'] = float(livro.find("span", {"class" : "minirating"}).get_text().split('-')[0].split()[0])

    #Total de avaliacoes - total ratings 
    dicionario_livros_g['total_avaliacoes'] = int((livro.find("a", {"href" : "#"}).get_text().split()[1]).replace(',',''))

    #Url
    dicionario_livros_g['url'] = 'https://www.goodreads.com'+livro.find('a', {'class':"bookTitle"}).get('href')



    #Paginas alemao - german pages
    url_g_2 = dicionario_livros_g['url']
    response_g_2 = urlopen(url_g_2)
    html_g_2 = response_g_2.read()
    soup_g_2 = BeautifulSoup(html_g_2)
    if soup_g_2.find("span", {"itemprop" : "numberOfPages" }):
        dicionario_livros_g['numero_paginas'] = int(soup_g_2.find("span", {"itemprop" : "numberOfPages" }).get_text().split()[0])
        
    
    
    itens_g.append(dicionario_livros_g)

In [276]:
# PT -> Criar dataframe oara cada gênero com as informacoes geradas a partir do webscraping da pagina do goodreads
# EN -> Create a dataframe for each book genre with  the information colected 


#business

df_b = pd.DataFrame(itens_b)
df_b = df_b[['titulo','autor','total_avaliacoes','nota','numero_paginas','url']]
df_b.dropna(inplace = True)
df_b.reset_index(inplace = True)
df_b.drop(['index'], axis =1)


#crime

df_c = pd.DataFrame(itens_c)
df_c = df_c[['titulo','autor','total_avaliacoes','nota','numero_paginas','url']]
df_c.dropna(inplace = True)
df_c.reset_index(inplace = True)
df_c.drop(['index'], axis =1)




#german 

df_g = pd.DataFrame(itens_g)
df_g = df_g[['titulo','autor','total_avaliacoes','nota','numero_paginas','url']]
df_g.dropna(inplace = True)
df_g.reset_index(inplace = True)
df_g.drop(['index'], axis =1)


,titulo,autor,total_avaliacoes,nota,numero_paginas,url
0,Faust: Der Tragödie Erster Teil,Johann Wolfgang von Goethe,2057,3.93,135.0,https://www.goodreads.com/book/show/1095059.Faust
1,Nathan der Weise,Gotthold Ephraim Lessing,1355,3.50,160.0,https://www.goodreads.com/book/show/1097229.Na...
2,The Metamorphosis,Franz Kafka,1276,3.83,201.0,https://www.goodreads.com/book/show/485894.The...
3,Die Physiker,Friedrich Dürrenmatt,983,4.04,95.0,https://www.goodreads.com/book/show/452220.Die...
4,Die Leiden des jungen Werthers,Johann Wolfgang von Goethe,950,3.68,160.0,https://www.goodreads.com/book/show/2075669.Di...
5,Woyzeck/Leonce und Lena,Georg Büchner,852,3.37,86.0,https://www.goodreads.com/book/show/162326.Woy...
6,Kleider machen Leute,Gottfried Keller,776,3.23,72.0,https://www.goodreads.com/book/show/1330198.Kl...
7,Wilhelm Tell,Friedrich Schiller,773,3.34,144.0,https://www.goodreads.com/book/show/942337.Wil...
8,Kabale und Liebe,Friedrich Schiller,760,3.22,143.0,https://www.goodreads.com/book/show/942335.Kab...
9,Emilia Galotti,Gotthold Ephraim Lessing,686,3.08,93.0,https://www.goodreads.com/book/show/1436901.Em...


In [278]:
# PT -> Criar um único dataframe a partir de cada um dos três dataframes criados anteriormente
# EN -> Create a unique dataframe with all the info from the previous three dataframes 


df_total = df_b.append(df_c)
df_total = df_total.append(df_g)
df_total.to_csv('df_total.csv')

In [ ]:
# PT -> Salvar cada dataframe criado em csv
# EN -> Salve each dataframe independently in a csv file 

df_b.to_csv('df_b.csv')
df_c.to_csv('df_c.csv')
df_g.to_csv('df_g.csv')

In [ ]:
# PT -> Final da primeira parte do projeto, etapa de aquisição dos dados com a utilização da biblioteca beautiful soup
# EN -> End of the first part of the project. Acquisition of the data using the lib beautiful soup 

In [414]:
# PT -> Obter dataframe salvo com o final da etapa anterior e formatar o mesmo 
# EN -> Read the csv file created at the end of the previous step and format it 


df_total = pd.read_csv('df_total.csv')
df_total.drop(['Unnamed: 0','index'], axis=1, inplace = True)


In [415]:
# PT -> Criar nova coluna com o genero do livro escrito e formatar campo de números 
# EN -> Create a new column for the book genre and format the numbers

df_total['categoria'] = 0
df_total.iloc[0:38,6] = 'business'
df_total.iloc[38:110,6] = 'crime'
df_total.iloc[110:,6] = 'alemao'
df_total['total_avaliacoes'] = df_total['total_avaliacoes'].astype('float')
df_total['nota'] = df_total['nota'].astype('float')
df_total['numero_paginas'] = df_total['numero_paginas'].astype('float')

In [416]:
# PT -> Primeira etapa da utilizacao da biblioteca pulp. Declarar a criação do problema de programação linear 
# EN -> First step using  the pulp lib. Create the linear programming problem. 

prob = LpProblem("Otimizando_qtd_livros_more_books", LpMaximize)

In [417]:
# PT -> Criar as variáveis de decisão do problema de programação linear  
# EN -> Create the decision variables of the linear programming problem  

#Variaveis de decisao livros business
variaveis_decisao_x = []
variaveis_decisao_y = []
variaveis_decisao_z = []
for index, linha in df_total.iterrows():
    if linha[6] == 'business':
        variavel_x = str('x' + str(index))
        variavel_x = LpVariable(str(variavel_x), lowBound = 0, upBound = 1, cat='Integer')
        variaveis_decisao_x.append(variavel_x)
    elif linha[6] == 'crime':
        variavel_y = str('y' + str(index))
        variavel_y = LpVariable(str(variavel_y), lowBound = 0, upBound = 1, cat='Integer')
        variaveis_decisao_y.append(variavel_y)
    else:
        variavel_z = str('z' + str(index))
        variavel_z = LpVariable(str(variavel_z), lowBound = 0, upBound = 1, cat='Integer')
        variaveis_decisao_z.append(variavel_z)

    
    
print(f'O problema tem {len(variaveis_decisao_x)} variaveis de decisao')
print(f'O problema tem {len(variaveis_decisao_y)} variaveis de decisao')
print(f'O problema tem {len(variaveis_decisao_z)} variaveis de decisao')

O problema tem 38 variaveis de decisao
O problema tem 72 variaveis de decisao
O problema tem 76 variaveis de decisao


In [418]:
# PT -> Na etapa de criar o problema já definimos que será um problema de maximização 
# EN -> Create the objective function of the LP problem. 


prob +=  lpSum(variaveis_decisao_x+variaveis_decisao_y+variaveis_decisao_z)

In [419]:
# PT -> Definir restrições do problema de programação linear. Restrições de tempo de leitura  
# EN -> Create the restrictions of the LP problem. Restriction of time used to read the books 


horas_leitura_semana = 4  #(30 min * 5 + 90 min)
paginas_lidas_hora = 50  



#Restricao tempo de leitura
total_paginas_para_ler = ''

#considerando os livros de business
for index, linha in df_total.iterrows():
    for i, variavel in enumerate(variaveis_decisao_x):
        if index == i:
            total_paginas_para_ler += linha['numero_paginas']*variavel
            
            
#Acrescentando os livros de crime
for index, linha in df_total.iterrows():
    for i, variavel in enumerate(variaveis_decisao_y):
        if index == i+38:
            total_paginas_para_ler += linha['numero_paginas']*variavel
            
            
#Acrescentando os livros de Deutsch
for index, linha in df_total.iterrows():
    for i, variavel in enumerate(variaveis_decisao_z):
        if index == i+110:
            total_paginas_para_ler += linha['numero_paginas']*variavel
            
            
#Calcular o total de horas para leitura no ano            
maximo_paginas_lidas = 52*horas_leitura_semana*paginas_lidas_hora

prob += (total_paginas_para_ler == maximo_paginas_lidas) 



In [420]:
#Restricao de numero minimo de paginas 50%business
# PT -> Definir restrições do problema de programação linear. Restriçção de quantidade mínima de páginas de livros de business  
# EN -> Create the restrictions of the LP problem. Restriction of minimum quantity of bussiness pages 

total_paginas_business_para_ler = ''

#considerando os livros de business
for index, linha in df_total.iterrows():
    for i, variavel in enumerate(variaveis_decisao_x):
        if index == i:
            total_paginas_business_para_ler += linha['numero_paginas']*variavel
            
            


prob += (total_paginas_business_para_ler >= 0.5* maximo_paginas_lidas) 

In [421]:
#Restricao de numero minimo de paginas 20%business
# PT -> Definir restrições do problema de programação linear. Restriçção de quantidade mínima de páginas de livros de alemão  
# EN -> Create the restrictions of the LP problem. Restriction of minimum quantity of german pages 

total_paginas_alemao_para_ler = ''

#considerando os livros de alemao
for index, linha in df_total.iterrows():
    for i, variavel in enumerate(variaveis_decisao_z):
        if index == i:
            total_paginas_alemao_para_ler += linha['numero_paginas']*variavel
            
            


prob += (total_paginas_alemao_para_ler >= 0.2* maximo_paginas_lidas) 

In [422]:
# PT -> Ver o problema completo  
# EN -> See the complete problem  

prob

Otimizando_qtd_livros_more_books:
MAXIMIZE
1*x0 + 1*x1 + 1*x10 + 1*x11 + 1*x12 + 1*x13 + 1*x14 + 1*x15 + 1*x16 + 1*x17 + 1*x18 + 1*x19 + 1*x2 + 1*x20 + 1*x21 + 1*x22 + 1*x23 + 1*x24 + 1*x25 + 1*x26 + 1*x27 + 1*x28 + 1*x29 + 1*x3 + 1*x30 + 1*x31 + 1*x32 + 1*x33 + 1*x34 + 1*x35 + 1*x36 + 1*x37 + 1*x4 + 1*x5 + 1*x6 + 1*x7 + 1*x8 + 1*x9 + 1*y100 + 1*y101 + 1*y102 + 1*y103 + 1*y104 + 1*y105 + 1*y106 + 1*y107 + 1*y108 + 1*y109 + 1*y38 + 1*y39 + 1*y40 + 1*y41 + 1*y42 + 1*y43 + 1*y44 + 1*y45 + 1*y46 + 1*y47 + 1*y48 + 1*y49 + 1*y50 + 1*y51 + 1*y52 + 1*y53 + 1*y54 + 1*y55 + 1*y56 + 1*y57 + 1*y58 + 1*y59 + 1*y60 + 1*y61 + 1*y62 + 1*y63 + 1*y64 + 1*y65 + 1*y66 + 1*y67 + 1*y68 + 1*y69 + 1*y70 + 1*y71 + 1*y72 + 1*y73 + 1*y74 + 1*y75 + 1*y76 + 1*y77 + 1*y78 + 1*y79 + 1*y80 + 1*y81 + 1*y82 + 1*y83 + 1*y84 + 1*y85 + 1*y86 + 1*y87 + 1*y88 + 1*y89 + 1*y90 + 1*y91 + 1*y92 + 1*y93 + 1*y94 + 1*y95 + 1*y96 + 1*y97 + 1*y98 + 1*y99 + 1*z110 + 1*z111 + 1*z112 + 1*z113 + 1*z114 + 1*z115 + 1*z116 + 1*z117 + 1*z11

In [429]:
# PT -> Rodar otimizaçãoe mostrars resultados
# EN -> Run the optimization and show the results

dict_solucao = {}


optimization_result = prob.solve()
assert optimization_result == LpStatusOptimal
print("Status:", LpStatus[prob.status])
print ("Individual decision_variables: ")
for v in prob.variables():
    dict_solucao[v.name]  = v.varValue
    print(v.name, "=", v.varValue)
    
print("O total de livros lidos sera de: ")  
print(prob.objective.value())

Status: Optimal
Individual decision_variables: 
x0 = 0.0
x1 = 1.0
x10 = 1.0
x11 = 1.0
x12 = 1.0
x13 = 0.0
x14 = 1.0
x15 = 1.0
x16 = 0.0
x17 = 0.0
x18 = 1.0
x19 = 1.0
x2 = 0.0
x20 = 1.0
x21 = 0.0
x22 = 0.0
x23 = 1.0
x24 = 1.0
x25 = 0.0
x26 = 0.0
x27 = 0.0
x28 = 0.0
x29 = 1.0
x3 = 0.0
x30 = 0.0
x31 = 1.0
x32 = 1.0
x33 = 1.0
x34 = 0.0
x35 = 0.0
x36 = 1.0
x37 = 0.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 0.0
x8 = 1.0
x9 = 1.0
y100 = 0.0
y101 = 0.0
y102 = 0.0
y103 = 0.0
y104 = 0.0
y105 = 0.0
y106 = 0.0
y107 = 0.0
y108 = 0.0
y109 = 0.0
y38 = 0.0
y39 = 0.0
y40 = 0.0
y41 = 0.0
y42 = 0.0
y43 = 0.0
y44 = 0.0
y45 = 0.0
y46 = 0.0
y47 = 0.0
y48 = 0.0
y49 = 0.0
y50 = 0.0
y51 = 0.0
y52 = 0.0
y53 = 0.0
y54 = 0.0
y55 = 1.0
y56 = 0.0
y57 = 1.0
y58 = 1.0
y59 = 0.0
y60 = 0.0
y61 = 0.0
y62 = 0.0
y63 = 0.0
y64 = 0.0
y65 = 0.0
y66 = 0.0
y67 = 0.0
y68 = 0.0
y69 = 0.0
y70 = 0.0
y71 = 0.0
y72 = 0.0
y73 = 0.0
y74 = 0.0
y75 = 0.0
y76 = 0.0
y77 = 0.0
y78 = 0.0
y79 = 0.0
y80 = 0.0
y81 = 0.0
y82 = 0.0
y83 = 0.0
y84 = 0.0
y8

In [435]:
# PT -> Reordenar os resultados de cada variável em relação ao nome dos livros e imprimir resultado  em um dataframe 
# EN -> Reorder the decision variables results and add this info at the dataframe 


variable_name = []
variable_value = []

for v in prob.variables():
    variable_name.append(v.name)
    variable_value.append(v.varValue)

df = pd.DataFrame({'variable': variable_name, 'value': variable_value})
for rownum, row in df.iterrows():
    value = re.findall(r'(\d+)', row['variable'])
    df.loc[rownum, 'variable'] = int(value[0])

df = df.sort_index(by='variable')

#append results
for rownum, row in df_total.iterrows():
    for results_rownum, results_row in df.iterrows():
        if rownum == results_row['variable']:
            df_total.loc[rownum, 'decision'] = results_row['value']

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)


In [442]:
# PT -> Salvar resultados em csv e html 
# EN -> Save the results in csv and html
df_total.to_csv('resultado_final.csv')
df_total.to_html('resultado_final.html')

In [449]:
df_total[df_total['decision'] == 1 ].to_csv('Lista_livros.csv')
df_total[df_total['decision'] == 1 ].reset_index().to_html('Lista_livros.html')

In [447]:
df_total[df_total['decision'] == 1 ]

,titulo,autor,total_avaliacoes,nota,numero_paginas,url,categoria,decision
1,How to Win Friends and Influence People,Dale Carnegie,672370.0,4.21,288.0,https://www.goodreads.com/book/show/4865.How_t...,business,1.0
4,The Tipping Point: How Little Things Can Make ...,Malcolm Gladwell,716317.0,3.98,301.0,https://www.goodreads.com/book/show/2612.The_T...,business,1.0
5,Rework,Jason Fried,143450.0,3.96,279.0,https://www.goodreads.com/book/show/6732019-re...,business,1.0
6,Start with Why: How Great Leaders Inspire Ever...,Simon Sinek,138964.0,4.08,256.0,https://www.goodreads.com/book/show/7108725-st...,business,1.0
8,The E-Myth Revisited: Why Most Small Businesse...,Michael E. Gerber,66216.0,4.03,269.0,https://www.goodreads.com/book/show/81948.The_...,business,1.0
9,The Innovator's Dilemma: The Revolutionary Boo...,Clayton M. Christensen,42188.0,4.02,286.0,https://www.goodreads.com/book/show/2615.The_I...,business,1.0
10,Freakonomics: A Rogue Economist Explores the H...,Steven D. Levitt,740131.0,3.98,320.0,https://www.goodreads.com/book/show/1202.Freak...,business,1.0
11,Think and Grow Rich,Napoleon Hill,238106.0,4.18,233.0,https://www.goodreads.com/book/show/30186948-t...,business,1.0
12,Made to Stick: Why Some Ideas Survive and Othe...,Chip Heath,78455.0,3.96,291.0,https://www.goodreads.com/book/show/69242.Made...,business,1.0
14,Blue Ocean Strategy: How to Create Uncontested...,W. Chan Kim,58029.0,3.96,240.0,https://www.goodreads.com/book/show/4898.Blue_...,business,1.0
